In [21]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime

# Download the website's HTML code
url = 'https://www.gewobag.de/fuer-mieter-und-mietinteressenten/mietangebote/'\
    '?bezirke_all=1&bezirke%5B%5D=charlottenburg-wilmersdorf&'\
    'bezirke%5B%5D=charlottenburg-wilmersdorf-charlottenburg&'\
    'bezirke%5B%5D=charlottenburg-wilmersdorf-grunewald&'\
    'bezirke%5B%5D=friedrichshain-kreuzberg&'\
    'bezirke%5B%5D=friedrichshain-kreuzberg-friedrichshain&'\
    'bezirke%5B%5D=friedrichshain-kreuzberg-kreuzberg&'\
    'bezirke%5B%5D=lichtenberg&'\
    'bezirke%5B%5D=lichtenberg-alt-hohenschoenhausen&'\
    'bezirke%5B%5D=lichtenberg-falkenberg&'\
    'bezirke%5B%5D=lichtenberg-fennpfuhl&'\
    'bezirke%5B%5D=lichtenberg-friedrichsfelde&'\
    'bezirke%5B%5D=marzahn-hellersdorf&'\
    'bezirke%5B%5D=marzahn-hellersdorf-marzahn&'\
    'bezirke%5B%5D=mitte&bezirke%5B%5D=mitte-gesundbrunnen&'\
    'bezirke%5B%5D=mitte-moabit&'\
    'bezirke%5B%5D=mitte-wedding&'\
    'bezirke%5B%5D=neukoelln&'\
    'bezirke%5B%5D=neukoelln-buckow&'\
    'bezirke%5B%5D=neukoelln-rudow&'\
    'bezirke%5B%5D=pankow&'\
    'bezirke%5B%5D=pankow-prenzlauer-berg&'\
    'bezirke%5B%5D=pankow-weissensee&'\
    'bezirke%5B%5D=reinickendorf&'\
    'bezirke%5B%5D=reinickendorf-hermsdorf&'\
    'bezirke%5B%5D=reinickendorf-tegel&'\
    'bezirke%5B%5D=reinickendorf-waidmannslust&'\
    'bezirke%5B%5D=spandau&'\
    'bezirke%5B%5D=spandau-falkenhagener-feld&'\
    'bezirke%5B%5D=spandau-hakenfelde&'\
    'bezirke%5B%5D=spandau-haselhorst&'\
    'bezirke%5B%5D=spandau-staaken&'\
    'bezirke%5B%5D=spandau-wilhelmstadt&'\
    'bezirke%5B%5D=steglitz-zehlendorf&'\
    'bezirke%5B%5D=steglitz-zehlendorf-lichterfelde&'\
    'bezirke%5B%5D=tempelhof-schoeneberg&'\
    'bezirke%5B%5D=tempelhof-schoeneberg-lichtenrade&'\
    'bezirke%5B%5D=tempelhof-schoeneberg-mariendorf&'\
    'bezirke%5B%5D=tempelhof-schoeneberg-marienfelde&'\
    'bezirke%5B%5D=tempelhof-schoeneberg-schoeneberg&'\
    'bezirke%5B%5D=treptow-koepenick&'\
    'bezirke%5B%5D=treptow-koepenick-alt-treptow&'\
    'bezirke%5B%5D=treptow-koepenick-niederschoeneweide&'\
    'bezirke%5B%5D=treptow-koepenick-oberschoeneweide&'\
    'nutzungsarten%5B%5D=wohnung&'\
    'gesamtmiete_von=&'\
    'gesamtmiete_bis=&'\
    'gesamtflaeche_von=&'\
    'gesamtflaeche_bis=&'\
    'zimmer_von=&'\
    'zimmer_bis=&'\
    'sort-by=recent'

response = requests.get(url)
html = response.content

now = datetime.now()

# Parse the HTML code with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Find the specific HTML tag or attribute that corresponds to the change you are looking for
empty = soup.find('div', {'class': 'empty-mietangebote'})

# Check if the element has changed
if empty is None:
    results = soup.find(string=re.compile("Suchergebnisse"))
    
    if results[0] == "1":
        results = "Es wurde 1 Suchergebnis gefunden \nTimestamp: " + now.strftime("%H:%M:%S")
    else: 
        results = "Es wurden " + results + " gefunden \nTimestamp: " + now.strftime("%H:%M:%S")
    #print(results + " gefunden!")
else:
    print("Keine Wohnungen vorhanden")

In [23]:
import smtplib
from email.message import EmailMessage

def email_alert (subject, body, to):
    msg = EmailMessage()
    msg.set_content(body)
    msg['subject'] = subject
    msg['to'] = to

    user = "immo.scraper.alert@gmail.com"

    msg['from'] = user

    password = "bndeucipumleyiad"
    
    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.starttls()
    server.login(user, password)
    server.send_message(msg)
    server.quit()

now = datetime.now()
email_alert("Wohnung Gefunden", results, "mail@olivervanvelzen.com")

In [22]:
import http.client, urllib

now = datetime.now()
conn = http.client.HTTPSConnection("api.pushover.net:443")
conn.request("POST", "/1/messages.json",
  urllib.parse.urlencode({
    "token": "ajbg2qtadunfbq8o6nqhu7tm5apvey",
    "user": "uoy8sg13a5hm6o5s87584ppcgtt6a9",
    "message": results,
  }), { "Content-type": "application/x-www-form-urlencoded" })
conn.getresponse()